In [39]:
'065676'

'065676'

In [1]:
# ! pip install jugaad_data, nsetools
from jugaad_data.nse import stock_df
from nsetools import Nse

from datetime import date, datetime, timedelta
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as fplt
import plotly.graph_objects as go
from plotly import colors as plotly_colors

from multiprocessing import Pool

import requests
from bs4 import BeautifulSoup
import random

from os import listdir
from os.path import isfile, join

import json
import warnings
import math
from random import sample

import backtrader


from helpers.candlestick import CandlePattern

# Defaults

In [7]:
CP = CandlePattern()

pd.set_option('display.max_rows', 100)

In [9]:
def get_stocks(name):
    try:
        df = stock_df(symbol=name, from_date = current_date - timedelta(days = 600), to_date = current_date, series="EQ").drop(drop,axis=1)
        df['DATE'] = pd.to_datetime(df['DATE'])
        ID, NAME, _ = all_stocks[name].split('_')
        save = f"./data/{ID}_{NAME}_{str(current_date)}.csv"
        df.to_csv(save,index=None)
    except Exception as e:
        print(e)
        pass


 
path = './data'
AS = AnalyseStocks(check_fresh = False)
all_stocks = AS.all_stocks

drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
current_date = date.today()
stocks = set(all_stocks.keys()) - set([i.split('_')[0] for i in listdir('./data')]) 

pool = Pool(4)
results = pool.map(get_stocks,stocks)
pool.close()
pool.join()

NameError: name 'AnalyseStocks' is not defined

# Code

In [2]:
class DataHandler:
    def __init__(self, data_path = './data', check_fresh = True):
        self.present = date.today()
        self.week_num = self.present.strftime("%W")
        
        self.data_path = data_path
        
        if check_fresh:
            self.__fresh()
        self.data = self.read_data()
        self.all_stocks = self.read_data()['all_stocks']
        
         
    def __fresh(self,):
        files = listdir(self.data_path)
        if not len(files):
            raise Exception(f"No CSV data files present at {self.data_path} Download new data for analysis")

        self.data = self.read_data()
        for file in files:
            key, _ , _ = file.split('_')
            self.data['all_stocks'][key] = file
        self.update_data(self.data)
  
    
    def read_data(self, path = './', file = 'data.json'):
        '''
        Write the data in json file
        args:
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file)) as f:
            return json.load(f)


    def update_data(self, updated_data:dict, path:str = './', file:str = 'data.json'):
        '''
        Update the data in the json file
        args:
            updated_data: Dictonary you want to update
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file), 'w') as f:
            json.dump(updated_data,f)
            return True 
    
    
    def open_live_stock_data(self,name:str):
        '''
        Open the fresh stock from the market
        args:
            name: ID of the stock given
        '''
        drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
        return stock_df(symbol=name, from_date = self.present - timedelta(days = 600), to_date = self.present, series="EQ").drop(drop,axis=1)
    
    
    def open_downloaded_stock(self,name:str):
        '''
        Open the Individual stock based on it's Official Term
        args:
            name: Name / ID given to the stock. Example, Infosys is "INFY"
        returns: DataFrame of that stock
        '''
        return pd.read_csv(join(self.data_path,self.all_stocks[name]))

In [3]:
class AnalyseStocks(DataHandler):
    def __init__(self, check_fresh = True):
        '''
        args:
            path: Path where all the stock files are saved
        '''
        super().__init__(check_fresh = check_fresh)
        self.eligible = {}
        self.registered_stocks = self.read_data()['registered_stocks']
        self.colors = self.read_data()['colors']
        self.rising = {}
    

    def is_ma_eligible(self, df, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH'),limit:float=11):
        '''
        Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            limit: Limit difference between average and low/high threshold for selecting stocks
        '''
        Date, Open, Close, Low, High = names
        Average = f'{str(mv)}-SMA'
        
        stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
        stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()
        
        last_traded = stocks.iloc[-1,:]
        low, high, open_ , close, avg, symbol = last_traded[Low],last_traded[High],last_traded[Open],last_traded[Close], last_traded[Average], last_traded['SYMBOL']
        
        if close < avg or close < open_ : # if red candle or below Average Line, Discard
            return False
        
        diff = min(abs(low - avg), (abs(high - avg)), abs(open_ - avg), abs(close - avg))
        if diff <= limit:
            self.rising[symbol] = stocks.iloc[-mv//2:-1,-1].mean() < avg  # whether the moving average is Upward or Downward according to last 44 days moving averages
            return {symbol : round(diff,2)}

        return False
    
    
    def BollingerBands(self,df, mv:int = 44):
        '''
        Calculate the Upper and Lower Bollinger Bands Given on a Moving Average Line
        args:
            df: Stocks Data
            mv: Moving Average Line value
        '''
        ticker = df.sort_index(ascending=False)

        sma = ticker['CLOSE'].rolling(window = mv).mean()
        std = ticker['CLOSE'].rolling(window = mv).std(0)

        upper_bb = sma + std * 2
        lower_bb = sma - std * 2

        ticker['Upper Bollinger'] = upper_bb
        ticker['Lower Bollinger'] = lower_bb

        return ticker.sort_index(ascending=True)
    
    
    def Ichimoku_Cloud(self,df):
        '''
        Get the values of Lines for Ichimoku Cloud
        args:
            df: Dataframe
        '''
        d = df.sort_index(ascending=False)

        # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
        period9_high = d['HIGH'].rolling(window=9).max()
        period9_low = d['LOW'].rolling(window=9).min()
        tenkan_sen = (period9_high + period9_low) / 2


        # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
        period26_high = d['HIGH'].rolling(window=26).max()
        period26_low = d['LOW'].rolling(window=26).min()
        kijun_sen = (period26_high + period26_low) / 2

        # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
        senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

        # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
        period52_high = d['HIGH'].rolling(window=52).max()
        period52_low = d['LOW'].rolling(window=52).min()
        senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

        # The most current closing price plotted 22 time periods behind (optional)
        chikou_span = d['CLOSE'].shift(-26) # Given at Trading View.

        d['blue_line'] = tenkan_sen
        d['red_line'] = kijun_sen
        d['cloud_green_line_a'] = senkou_span_a
        d['cloud_red_line_b'] = senkou_span_b
        d['lagging_line'] = chikou_span
        return d.sort_index(ascending=True)
    
    
    def _is_ichi(self,df, index:str = 'nifty_50'):
        '''
        Get All available IchiMoku available stocks
        args:
            index: Index to Open From
        '''
        count  = 0
        current = df.iloc[0,:]
        if current['cloud_green_line_a'] < current['LOW'] and current['cloud_red_line_b'] < current['LOW']: # Cloud Below
            count += 1
            if df.loc[26,'lagging_line'] > df.loc[26,'HIGH']: # Lagging Line
                count += 1
                if df.loc[1,'blue_line'] <= df.loc[1,'red_line'].min() and current['blue_line'] >= current['red_line']: # Cross Over
                    count += 1
        return count
    
    
    def update_eligible(self):
        '''
        Save all Eligible stocks for the current week
        '''
        for key in self.registered_stocks:
            result = self.is_ma_eligible(self.open_downloaded_stock(key))
            if result:
                self.eligible.update(result)
        return self.eligible
            
    
    def plot_candlesticks(self,df, names = ('DATE','OPEN','CLOSE','LOW','HIGH'), mv:list = [44,100,200]):
        '''
        Plot a candlestick on a given dataframe
        args:
            df: DataFrame
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            mv: Moving Averages
        '''
        stocks = df.copy()
        Date, Open, Close, Low, High = names
        colors = sample(self.colors,len(mv))
        stocks.sort_index(ascending=False, inplace = True)  # Without reverse, recent rolling mean will be either NaN or equal to the exact value
    

        candle = go.Figure(data = [go.Candlestick(x = stocks[Date], name = 'Trade',
                                                       open = stocks[Open], 
                                                       high = stocks[High], 
                                                       low = stocks[Low], 
                                                       close = stocks[Close]),])
        for i in range(len(mv)):
            stocks[f'{str(mv[i])}-SMA'] = stocks[Close].rolling(mv[i], min_periods = 1).mean()
            candle.add_trace(go.Scatter(name=f'{str(mv[i])} MA',x=stocks[Date], y=stocks[f'{str(mv[i])}-SMA'], 
                                             line=dict(color=colors[i], width=1.1)))

        candle.update_xaxes(
            title_text = 'Date',
            rangeslider_visible = True,
            rangeselector = dict(
                buttons = list([
                    dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                    dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                    dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                    dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                    dict(step = 'all')])))

        candle.update_layout(autosize = False, width = 1400, height = 600,
                             title = {'text': f"{stocks['SYMBOL'][0]} | {self.all_stocks[stocks['SYMBOL'][0]]}",'y':0.97,'x':0.5,
                                      'xanchor': 'center','yanchor': 'top'},
                             margin=dict(l=30,r=30,b=30,t=30,pad=2),
                             paper_bgcolor="lightsteelblue",)

        candle.update_yaxes(title_text = 'Price in Rupees', tickprefix = u"\u20B9" ) # Rupee symbol
        candle.show()

# Investing

In [63]:
class Investing(AnalyseStocks):
    def __init__(self,rolling_mean:int=44):
        '''
        args:
            rolling_mean: Rolling Simple Mean to calulate
        '''
        super().__init__()
        self.rm = rolling_mean
        self._eligible = None
        self.data = self.read_data()
        self.indices = {'nifty_50': 'Nifty 50','nifty_100':'Nifty 100','nifty_200':'Nifty 200','nifty_500':'Nifty 200'}
        self.all_ichi = None
        self.picked = None
        self._old_budget = -1
      

    def _get_all_ichi(self,budget:float, index:str='nifty_500', refit = False):
        '''
        Get all Stocks who are almost perfect for Ichimoku execution
        args:
            budget: Your Budget
            index: Which Index to Search
        '''
        if self.all_ichi and  (not refit):
            return self.all_ichi
        
        all_ichi = {}
        data = self.data[index] if index else self.all_stocks
        for name in data:
            df = self.Ichimoku_Cloud(self.open_downloaded_stock(name))
            count = self._is_ichi(df,index) 
            if count and df.loc[0,'HIGH'] < budget:
                all_ichi[name] = count
                
        self.all_ichi = all_ichi
        return self.all_ichi
    
    
    def highlight_falling(self, s, column:str):
        '''
        Highlight The rows where average is falling
        args:
            s: Series
            column: Column name(s)
        '''
        is_max = pd.Series(data=False, index=s.index)
        is_max[column] = s.loc[column] == True
        return ['' if is_max.any() else 'background-color: #f7a8a8' for v in is_max]
    
    
    def get_index(self,symbol:str):
        '''
        Get the Index of the symbol from nifty 50,100,200,500
        args:
            symbol: Name /  ID od the company on NSE
        '''
        for index in self.indices.keys():
            if symbol in self.data[index]:
                return self.indices[index]
        return 'Other'
    
    
    def calculate(self, budget, High:str = 'HIGH', Close:str = 'CLOSE', delta:float = 1, nifty:str = 'nifty_500', show_only:bool=True):
        '''
        Pick Stocks based on all available and which are within your budget
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            show_only: If you want to see formatted part only
            refit: Whether to recalculate stats again
        ''' 
        refit = True if self._old_budget < budget else False
        self._old_budget = budget
        ichi = self._get_all_ichi(budget ,refit = refit)
        
        if (not self._eligible) or refit:
            self._eligible  = self.update_eligible()
            
        keys = set(self._eligible.keys()).intersection(set(self.data[nifty])) if nifty else list(self._eligible.keys()) 
        values = []
        for key in keys:
            try:
                df = self.open_downloaded_stock(key)
            except Exception as e:
                print('Exception Opening: ',key)
                
            if df.loc[0,High] + delta > budget:
                del self._eligible[key]
                
            else:
                values.append(df.iloc[0,:])
                
        columns = df.columns
        df = pd.DataFrame(values,columns=columns,index = range(len(values)))
        df = df.merge(pd.DataFrame({'SYMBOL':self._eligible.keys(), 'Diff':self._eligible.values()}),on='SYMBOL')
        
        df['Rising'] = df['SYMBOL'].apply(lambda x: self.rising[x]) # Get Rising or Falling
        df['Ichi'] = df['SYMBOL'].apply(lambda x: ichi[x] if self._get_all_ichi(budget).get(x) else 0)
        df['Candle Type'] = df.apply(lambda row: CP.find_name(row['OPEN'],row['CLOSE'],row['LOW'],row['HIGH']), axis=1)
        df['Index'] = df['SYMBOL'].apply(lambda x: self.get_index(x)) # Get Rising or Falling

        self.picked = df.sort_values('Diff',ascending = True)
        if show_only:
            return self.picked.style.apply(self.highlight_falling, column=['Rising'], axis=1) # set style
        else:
            return self.picked
        
        
    def show_full_stats(self, budget, risk, High = 'HIGH', Close = 'CLOSE', delta:float=1, nifty:str = 'nifty_500',):
        '''
        Show Extra Stats
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            risk: How much risk you want to take
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            refit: Whether to recalculate the stats or not
        '''
        self.picked = self.calculate(budget, High, Close, delta, nifty, show_only = False)
        
        expec_change = []
        max_risk = []
        
        pic = self.picked.copy()
        for index in pic.index:
            name = pic.loc[index,'SYMBOL']
            result = self.get_particulars(name,budget,risk)
            if result:
                expec_change.append(result['Target %'])
                max_risk.append(result['Max loss on this config'])
            else:
                expec_change.append(np.inf)
                max_risk.append(np.inf)

        pic['Expected Change %'] = expec_change
        pic['Max Config Risk'] = max_risk
        return pic
        
        
    def get_particulars(self, name, budget:float, max_loss_capacity:float, risk_to_reward_ratio:float=2, Low:str = 'LOW', High:str = 'HIGH', delta:float = 0.3, plot_candle:bool = False):
        '''
        Display the particulars of a trade before buying
        args:
            name: name of the particular stock
            loss_capacity: How much loss you can survive at the end of day PER SHARE. Total capacity will be No of shares * per share loss capacity
            risk_to_reward_ratio: How much profit you want to have. It is twice of loss_capacity per share for 44 Moving average
            budget : How much you have for investing purpose
            Low: Column name which describes LOW of the previous trade
            High =  Column name which describes High of the previous trade
            delta: A min amount above which you'll buy
            plot_candle: Plot the candlestick for the stock
        '''   
        df = self.open_downloaded_stock(name)
        if risk_to_reward_ratio > 2:
            warnings.warn(f"Don't be greedy with risk to reward ratio of {risk_to_reward_ratio}. Stick to system")
            
        if  max_loss_capacity > 0.011 * budget:
            warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total PORTFOLIO. Going Like this, you'll lose Rs {max_loss_capacity*15} in 15 Trades. Try keeping it less than 1.1% @ Rs {round(budget*0.011,2)}.")
            
        risk = max_loss_capacity # per share
        entry = df.loc[0,High] + delta # Last Day MAX + Delta
        stop_loss = min(df.loc[:2,Low].values) # Min of the last 2 
        max_loss = entry - stop_loss
        stop_loss_perc = round((max_loss / entry) *100,2)
        diff = entry - stop_loss
        quantity = min(risk // diff , budget // entry)
        profit = risk_to_reward_ratio * diff
        profit_perc = round((profit/entry)*100,2)
        target = round(entry + profit,2)
        
        
        if plot_candle:
            AnalyseStocks().plot_candlesticks(df)
    
        if budget < entry:
            warnings.warn(f"Budget for {name} should be a minimum of Rs. {entry}")
            return None
        
        if quantity < 1:
            r = round(risk + (diff - risk), 2)
            warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
            return None
            
        return {'Buying Price':round(entry,2),'Stop-Loss %': stop_loss_perc,'Target %':profit_perc,'Quantity':quantity,'Stop-Loss Price':stop_loss,'Trigger Price':target,
                'Risk Per Share':round(diff,2),'Profit Per Share':round(profit,2),'Max loss on this config':round(quantity*diff,2),
                'Max Gain on this config': round(quantity*profit,2),'Price To Profit Ratio': round(entry / (diff*2 ),2), 'Index':self.get_index(name),}
    
In = Investing()

In [75]:
df = In.show_full_stats(15000,150,nifty='nifty_500')

In [76]:
df.sort_values('Expected Change %')

,DATE,OPEN,HIGH,LOW,LTP,CLOSE,52W H,52W L,SYMBOL,Diff,Rising,Ichi,Candle Type,Index,Expected Change %,Max Config Risk
19,2021-07-27,3073.65,3112.65,3050.85,3082.85,3077.75,3294.70,1584.00,GALAXYSURF,10.14,True,0,Doji,Nifty 200,4.01,124.90
6,2021-07-27,709.85,717.90,703.30,709.55,710.65,773.00,200.10,JSWSTEEL,7.86,False,0,Doji,Nifty 50,4.79,137.60
12,2021-07-27,146.00,149.40,146.00,148.00,148.40,157.30,76.10,INDHOTEL,4.46,True,0,Shooting Star,Nifty 200,6.28,145.70
10,2021-07-27,424.15,433.25,424.00,429.20,429.95,441.95,175.50,SBIN,2.01,True,0,Shooting Star,Nifty 50,6.48,140.50
5,2021-07-27,85.15,86.60,84.20,86.15,85.95,89.70,28.05,JAMNAAUTO,0.29,True,1,Hammer,Nifty 200,6.67,147.90
16,2021-07-27,49.60,50.40,49.50,50.20,50.10,58.00,31.35,NETWORK18,0.18,True,0,Normal,Nifty 200,6.71,149.60
9,2021-07-27,56.85,58.25,56.40,57.65,57.40,62.90,18.50,IDFC,1.01,True,2,Shooting Star,Nifty 200,7.86,149.50
4,2021-07-27,174.75,178.35,173.40,175.00,175.00,187.30,119.35,JYOTHYLAB,10.32,True,0,Doji,Nifty 200,8.00,143.00
21,2021-07-27,217.15,224.90,217.05,222.00,223.05,231.65,117.00,ABFRL,7.18,True,0,Shooting Star,Nifty 200,8.97,141.40
11,2021-07-27,2230.25,2320.00,2228.25,2300.00,2297.05,2580.00,659.00,HEG,8.25,False,0,Shooting Star,Nifty 200,10.27,119.15
